# Spiking Neural Network (SNN) - PyTorch Translation

This notebook translates a TensorFlow/Keras SNN implementation to PyTorch.
Each cell includes detailed explanations of the translation process and significance.

## Cell 1: Import Libraries

**Translation Details:**
- `tensorflow` → `torch` (main deep learning framework)
- `keras` → `torch.nn` (neural network modules)
- `numpy` → kept the same (numerical operations)
- `matplotlib.pyplot` → kept the same (visualization)

**Significance:**
PyTorch uses a more explicit, imperative programming style compared to Keras's declarative approach.
We'll need to handle autograd and custom gradients differently.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

# Set device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Cell 2: Define Surrogate Gradient Spike Function

**Translation Details:**
- TensorFlow's `@tf.custom_gradient` → PyTorch's `torch.autograd.Function`
- `tf.greater()` → simple comparison `>`
- `tf.cast()` → `.float()`
- TensorFlow's nested gradient function → PyTorch's `backward()` method

**Significance:**
This is the **CRITICAL** component for training SNNs:
- **Forward pass:** Uses Heaviside step function (hard threshold) to generate actual spikes
- **Backward pass:** Uses a smooth "fast sigmoid" surrogate gradient to enable gradient flow
- **Why needed:** The Heaviside function has zero gradient everywhere (except undefined at threshold),
  making it impossible to train with standard backpropagation. The surrogate gradient solves this!

**Mathematical Background:**
- Forward: spike = 1 if v > threshold, else 0
- Backward: ∂spike/∂v ≈ 1/(1 + |α(v - threshold)|)² (fast sigmoid derivative)
- α = 10 controls the steepness of the surrogate gradient

In [ ]:
class SpikeFunction(torch.autograd.Function):
    """
    Custom autograd function for spike generation with surrogate gradient.
    
    This is the key innovation that makes SNNs trainable with backprop!
    """
    
    @staticmethod
    def forward(ctx, v, threshold):
        """
        Forward pass: Generate actual spikes using hard threshold (Heaviside).
        
        Args:
            v: Membrane potential tensor [batch, neurons]
            threshold: Spike threshold value
            
        Returns:
            spikes: Binary spike tensor (1 if v > threshold, else 0)
        """
        # Save for backward pass
        ctx.save_for_backward(v)
        ctx.threshold = threshold
        
        # Generate spikes: 1 if voltage exceeds threshold, 0 otherwise
        spikes = (v > threshold).float()
        return spikes
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        Backward pass: Use fast sigmoid surrogate gradient.
        
        This is what enables training! We replace the non-differentiable
        Heaviside function with a smooth approximation during backprop.
        
        Args:
            grad_output: Gradient flowing back from next layer
            
        Returns:
            grad_v: Gradient with respect to membrane potential
            grad_threshold: None (threshold is constant)
        """
        v, = ctx.saved_tensors
        threshold = ctx.threshold
        
        # Surrogate gradient parameters
        alpha = 10.0  # Steepness of surrogate gradient
        
        # Compute distance from threshold
        v_shifted = v - threshold
        
        # Fast sigmoid surrogate gradient: 1 / (1 + |alpha * (v - threshold)|)^2
        surrogate_grad = 1.0 / (1.0 + torch.abs(alpha * v_shifted)) ** 2
        
        # Apply chain rule: multiply by incoming gradient
        grad_v = grad_output * surrogate_grad
        
        # Threshold is constant, so no gradient
        return grad_v, None

# Create a functional interface for easier use
spike_fn = SpikeFunction.apply

print("✓ Surrogate gradient spike function defined!")
print("Forward: Hard threshold (Heaviside step function)")
print("Backward: Smooth fast sigmoid gradient (α=10)")

## Cell 3: Define LIF (Leaky Integrate-and-Fire) Layer

**Translation Details:**
- `keras.layers.Layer` → `nn.Module`
- `self.add_weight()` → `nn.Parameter()` or `nn.Linear()`
- `tf.while_loop()` → standard Python `for` loop (PyTorch handles dynamic graphs automatically)
- `tf.TensorArray` → Python list (PyTorch is more flexible with dynamic operations)
- `build()` method → handled in `__init__()` in PyTorch

**Significance:**
The LIF neuron is a simplified biological neuron model:
1. **Leaky Integration:** Membrane potential decays over time (models ion channels)
2. **Input Current:** Weighted sum of inputs charges the neuron
3. **Spike Generation:** When voltage crosses threshold, neuron fires
4. **Reset:** After spiking, voltage resets to zero (refractory period)

**Equation:** dv/dt = -v/τ + I_in
- τ (tau): time constant controlling decay rate
- Discrete approximation: v = v*(1-1/τ) + I_in

In [ ]:
class LIFLayer(nn.Module):
    """
    Leaky Integrate-and-Fire neuron layer with surrogate gradient training.
    
    This layer simulates biological spiking neurons that:
    - Integrate input over time (like dendrites collecting signals)
    - Leak charge gradually (like ion channels)
    - Fire spikes when voltage exceeds threshold (like axon action potentials)
    - Reset after spiking (like refractory period)
    """
    
    def __init__(self, input_dim, output_dim, tau=20.0, threshold=1.0):
        """
        Initialize the LIF layer.
        
        Args:
            input_dim: Number of input features
            output_dim: Number of neurons in this layer
            tau: Membrane time constant (higher = slower decay)
            threshold: Voltage threshold for spike generation
        """
        super(LIFLayer, self).__init__()
        
        # Store neuron parameters
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.tau = tau
        self.threshold = threshold
        
        # Create fully connected layer (weight matrix + bias)
        # PyTorch's Linear layer handles both W and b
        self.fc = nn.Linear(input_dim, output_dim)
        
        # Initialize weights using Xavier/Glorot uniform (same as TensorFlow default)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)
    
    def forward(self, x):
        """
        Forward pass: Simulate LIF neuron dynamics over time.
        
        Args:
            x: Input spike trains [batch, time_steps, input_dim]
            
        Returns:
            Output spike trains [batch, time_steps, output_dim]
        """
        batch_size, time_steps, _ = x.shape
        
        # Initialize membrane potential to zero
        v = torch.zeros(batch_size, self.output_dim, device=x.device)
        
        # Store output spikes for each time step
        spikes_list = []
        
        # Simulate neuron dynamics over time
        # PyTorch handles dynamic graphs, so we can use regular for loops!
        for t in range(time_steps):
            # Extract input at current time step
            x_t = x[:, t, :]  # Shape: [batch, input_dim]
            
            # Compute input current: I = W*x + b
            i_in = self.fc(x_t)  # Shape: [batch, output_dim]
            
            # Update membrane potential with leaky integration
            # v(t+1) = v(t) * (1 - 1/tau) + I_in(t)
            # The (1 - 1/tau) term causes exponential decay
            v = v * (1.0 - 1.0/self.tau) + i_in
            
            # Generate spikes using surrogate gradient function
            # Forward: hard threshold, Backward: smooth gradient
            spike = spike_fn(v, self.threshold)
            
            # Reset membrane potential for neurons that spiked
            # v = v * (1 - spike) sets spiking neurons to 0
            v = v * (1.0 - spike)
            
            # Store spikes for this time step
            spikes_list.append(spike)
        
        # Stack spikes along time dimension
        # List of [batch, output_dim] → [batch, time_steps, output_dim]
        spikes = torch.stack(spikes_list, dim=1)
        
        return spikes

print("✓ LIF Layer defined!")
print("Key features:")
print("  - Leaky integration (exponential decay with tau={:.1f})".format(20.0))
print("  - Spike generation at threshold={:.1f}".format(1.0))
print("  - Hard reset after spiking")
print("  - Surrogate gradient for backprop")

## Cell 4: Build 2-Layer SNN Model

**Translation Details:**
- Keras functional API → PyTorch `nn.Module` with `nn.Sequential`
- `keras.Input()` → handled implicitly in PyTorch's forward pass
- `keras.Model()` → custom `nn.Module` class

**Significance:**
This creates a simple feedforward SNN with:
- **Input layer:** Receives spike-encoded MNIST images [batch, 20, 784]
- **Hidden layer:** 128 LIF neurons for feature extraction
- **Output layer:** 10 LIF neurons (one per digit class)

The network processes temporal information (20 time steps) to classify digits.
Unlike standard ANNs, SNNs process sequences of spikes over time!

In [ ]:
class SpikingNN(nn.Module):
    """
    2-layer Spiking Neural Network for MNIST classification.
    
    Architecture:
    Input [784] → LIF Hidden [128] → LIF Output [10]
    
    All layers process spike trains over time (temporal coding).
    """
    
    def __init__(self, input_dim=784, hidden_dim=128, output_dim=10, tau=20.0, threshold=1.0):
        """
        Initialize the spiking neural network.
        
        Args:
            input_dim: Input feature dimension (784 for flattened MNIST)
            hidden_dim: Number of neurons in hidden layer
            output_dim: Number of output classes
            tau: Membrane time constant for all neurons
            threshold: Spike threshold for all neurons
        """
        super(SpikingNN, self).__init__()
        
        # First LIF layer (input → hidden)
        self.lif1 = LIFLayer(input_dim, hidden_dim, tau=tau, threshold=threshold)
        
        # Second LIF layer (hidden → output)
        self.lif2 = LIFLayer(hidden_dim, output_dim, tau=tau, threshold=threshold)
    
    def forward(self, x):
        """
        Forward pass through the spiking network.
        
        Args:
            x: Input spike trains [batch, time_steps, input_dim]
            
        Returns:
            Output spike trains [batch, time_steps, output_dim]
        """
        # Pass through first LIF layer
        h = self.lif1(x)  # [batch, time_steps, hidden_dim]
        
        # Pass through second LIF layer
        out = self.lif2(h)  # [batch, time_steps, output_dim]
        
        return out

# Create model instance
model = SpikingNN(input_dim=784, hidden_dim=128, output_dim=10).to(device)

# Print model summary
print("✓ Spiking Neural Network created!")
print("\nModel Architecture:")
print(model)
print("\nTotal parameters:", sum(p.numel() for p in model.parameters()))
print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

## Cell 5: Rate Encoding Function

**Translation Details:**
- `np.random.rand()` → kept the same (NumPy is fine for preprocessing)
- `np.newaxis` → same in NumPy
- Logic remains identical

**Significance:**
**Rate coding** converts continuous values to spike trains:
- Higher pixel intensity → Higher firing rate (more spikes)
- Lower pixel intensity → Lower firing rate (fewer spikes)

**How it works:**
1. Generate random numbers uniformly in [0, 1] for each pixel at each time step
2. If random < (pixel_value * max_rate), generate a spike
3. Result: Pixel value of 0.8 → ~80% chance of spike per time step

**Example:** 
- Dark pixel (0.1) → ~1-2 spikes over 20 time steps
- Bright pixel (0.9) → ~14-16 spikes over 20 time steps

This is one way neurons encode information in the brain!

In [ ]:
def rate_encode(data, time_steps, max_rate=1.0):
    """
    Convert static data to spike trains using rate coding.
    
    Rate coding: Encode continuous values as firing rates (spikes/time).
    Higher values → More spikes (mimics biological neural coding).
    
    Args:
        data: Input data [batch, features] with values in [0, 1]
        time_steps: Number of time steps for spike train
        max_rate: Maximum firing rate (probability per time step)
        
    Returns:
        Spike trains [batch, time_steps, features] with binary spikes (0 or 1)
    """
    # Ensure data is in valid range [0, 1]
    data = np.clip(data, 0, 1)
    
    # Generate random numbers for probabilistic spike generation
    # Shape: [batch, time_steps, features]
    rand = np.random.rand(data.shape[0], time_steps, data.shape[1])
    
    # Generate spike if random < pixel_value * max_rate
    # data[:, np.newaxis, :] broadcasts to [batch, time_steps, features]
    # Higher pixel values → Higher spike probability
    spikes = (rand < data[:, np.newaxis, :] * max_rate).astype(np.float32)
    
    return spikes

print("✓ Rate encoding function defined!")
print("\nExample: pixel value 0.8 with max_rate=1.0 over 20 time steps")
print("  → Expected ~16 spikes (0.8 * 20 = 16)")
print("  → Each time step has 80% chance of spike")

## Cell 6: Load and Prepare MNIST Dataset

**Translation Details:**
- `keras.datasets.mnist` → `torchvision.datasets.MNIST`
- `keras.utils.to_categorical()` → PyTorch keeps labels as integers (cross entropy handles it)
- Manual normalization → same approach

**Significance:**
MNIST preprocessing steps:
1. **Flatten:** 28×28 images → 784-dimensional vectors
2. **Normalize:** Pixel values [0, 255] → [0, 1] (helps with training stability)
3. **Subset:** Use 1000 samples for faster training/testing
4. **One-hot encoding:** Convert labels to probability distributions

**Note:** We keep labels as one-hot for compatibility with the original implementation,
though PyTorch typically uses integer labels.

In [ ]:
# Import torchvision for MNIST dataset
from torchvision import datasets, transforms

# Load MNIST dataset
# Note: torchvision automatically downloads to './data' directory
train_dataset = datasets.MNIST(root='./data', train=True, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, download=True)

# Extract data as numpy arrays
x_train = train_dataset.data.numpy()
y_train = train_dataset.targets.numpy()
x_test = test_dataset.data.numpy()
y_test = test_dataset.targets.numpy()

# Flatten images from 28×28 to 784-dimensional vectors
x_train = x_train.reshape(-1, 784).astype('float32')  # [60000, 784]
x_test = x_test.reshape(-1, 784).astype('float32')    # [10000, 784]

# Normalize pixel values to [0, 1] range
# Dividing by 255 (max pixel value) scales to unit interval
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoding
# Example: label 3 → [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
def to_categorical(y, num_classes=10):
    """Convert integer labels to one-hot encoded vectors."""
    one_hot = np.zeros((y.shape[0], num_classes))
    one_hot[np.arange(y.shape[0]), y] = 1
    return one_hot.astype('float32')

y_train = to_categorical(y_train, 10)  # [60000, 10]
y_test = to_categorical(y_test, 10)    # [10000, 10]

# Use smaller subset for faster training (you can increase this)
num_samples = 1000
x_train = x_train[:num_samples]
y_train = y_train[:num_samples]

print(f"✓ MNIST dataset loaded and preprocessed!")
print(f"\nTraining data shape: {x_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test data shape: {x_test.shape}")
print(f"Test labels shape: {y_test.shape}")
print(f"\nData range: [{x_train.min():.2f}, {x_train.max():.2f}]")

## Cell 7: Encode Data as Spike Trains

**Translation Details:**
- Same logic as TensorFlow version
- No translation needed (uses NumPy)

**Significance:**
This is where we convert static images to temporal spike patterns!

**Before encoding:** Each image is 784 continuous values in [0, 1]
**After encoding:** Each image is 20 time steps of 784 binary spikes

**Visualization:**
```
Original pixel (value=0.8):
[0.8]

After rate encoding (20 time steps):
t=0: [1]
t=1: [1]
t=2: [0]  ← No spike (random < 0.8 failed)
t=3: [1]
...
Total: ~16 spikes out of 20 time steps (80% firing rate)
```

**Why 20 time steps?**
- Trade-off between temporal resolution and computational cost
- More time steps → Better encoding but slower training
- 20 is a reasonable balance for MNIST

In [ ]:
# Define temporal parameters
time_steps = 20  # Number of simulation time steps

# Convert training data to spike trains using rate coding
print("Encoding training data...")
x_train_spikes = rate_encode(x_train, time_steps, max_rate=0.8)
print(f"✓ Training spike trains shape: {x_train_spikes.shape}")
print(f"  [batch, time_steps, features] = {x_train_spikes.shape}")

# Convert test data to spike trains
print("\nEncoding test data...")
x_test_spikes = rate_encode(x_test, time_steps, max_rate=0.8)
print(f"✓ Test spike trains shape: {x_test_spikes.shape}")

# Verify spike encoding
avg_spikes_per_pixel = x_train_spikes.mean(axis=1).mean()  # Average across time and batch
print(f"\nAverage spikes per pixel: {avg_spikes_per_pixel:.3f}")
print(f"Expected: ~{0.8 * x_train.mean():.3f} (max_rate * avg_pixel_value)")

# Convert to PyTorch tensors
x_train_spikes = torch.FloatTensor(x_train_spikes)
y_train = torch.FloatTensor(y_train)
x_test_spikes = torch.FloatTensor(x_test_spikes)
y_test = torch.FloatTensor(y_test)

## Cell 8: Define Loss Function and Metrics

**Translation Details:**
- Custom TensorFlow loss function → Custom PyTorch function
- `tf.reduce_sum()` → `torch.sum()`
- `tf.nn.softmax()` → `F.softmax()`
- `keras.losses.categorical_crossentropy()` → `F.cross_entropy()`

**Significance:**
**Spike-based Classification:**
Standard ANNs output probabilities directly. SNNs output spike trains!

**How we convert spikes to predictions:**
1. **Sum spikes over time:** Count total spikes per output neuron
2. **Apply softmax:** Convert spike counts to probability distribution
3. **Compute cross-entropy:** Compare to true labels

**Example:**
```
Output spikes for digit 3:
Neuron 0: [0,0,1,0,0,...] → 3 total spikes
Neuron 1: [0,1,0,0,1,...] → 5 total spikes
Neuron 2: [0,0,0,1,0,...] → 4 total spikes
Neuron 3: [1,1,1,1,1,...] → 18 total spikes ← HIGHEST!
...

Spike counts: [3, 5, 4, 18, 2, 1, 3, 4, 2, 1]
After softmax: [0.01, 0.02, 0.01, 0.92, 0.00, ...] ← Neuron 3 has highest probability
Prediction: Class 3 ✓
```

**Why sum over time?**
- Integrates temporal information
- Neuron that fires most frequently over the trial period wins
- Mimics rate coding in biological neurons

In [ ]:
def spike_categorical_crossentropy(y_pred, y_true):
    """
    Loss function for spike-based classification.
    
    Converts spike trains to class probabilities by:
    1. Summing spikes over time (rate coding)
    2. Applying softmax normalization
    3. Computing cross-entropy loss
    
    Args:
        y_pred: Predicted spike trains [batch, time_steps, num_classes]
        y_true: True labels (one-hot) [batch, num_classes]
        
    Returns:
        Scalar loss value
    """
    # Sum spikes across time dimension to get total spike counts
    spike_counts = torch.sum(y_pred, dim=1)  # [batch, num_classes]
    
    # Apply softmax to convert spike counts to probabilities
    # This normalizes so that probabilities sum to 1
    probs = F.softmax(spike_counts, dim=-1)
    
    # Compute categorical cross-entropy loss
    # -sum(y_true * log(y_pred))
    loss = F.cross_entropy(probs, y_true)
    
    return loss


def spike_categorical_accuracy(y_pred, y_true):
    """
    Accuracy metric for spike-based classification.
    
    Args:
        y_pred: Predicted spike trains [batch, time_steps, num_classes]
        y_true: True labels (one-hot) [batch, num_classes]
        
    Returns:
        Accuracy as a percentage
    """
    # Sum spikes over time to get spike counts per class
    spike_counts = torch.sum(y_pred, dim=1)  # [batch, num_classes]
    
    # Get predicted class (neuron with most spikes)
    y_pred_class = torch.argmax(spike_counts, dim=-1)  # [batch]
    
    # Get true class from one-hot labels
    y_true_class = torch.argmax(y_true, dim=-1)  # [batch]
    
    # Compute accuracy
    correct = (y_pred_class == y_true_class).float()
    accuracy = correct.mean() * 100.0  # Convert to percentage
    
    return accuracy

print("✓ Loss function and metrics defined!")
print("\nLoss: Spike-based categorical cross-entropy")
print("  1. Sum spikes over time → spike counts")
print("  2. Apply softmax → probabilities")
print("  3. Compute cross-entropy loss")
print("\nMetric: Spike-based accuracy")
print("  - Neuron with most spikes wins")

## Cell 9: Training Setup

**Translation Details:**
- `model.compile()` → Manual training loop in PyTorch
- `keras.optimizers.Adam()` → `torch.optim.Adam()`
- `model.fit()` → Custom training loop with DataLoader

**Significance:**
PyTorch requires more explicit control over training:
1. **DataLoader:** Handles batching and shuffling
2. **Optimizer:** Updates weights based on gradients
3. **Training loop:** Manually iterate through epochs and batches

**Training process:**
```
For each epoch:
  For each batch:
    1. Forward pass: model(x) → predictions
    2. Compute loss: loss_fn(predictions, targets)
    3. Backward pass: loss.backward() → compute gradients
    4. Update weights: optimizer.step()
    5. Reset gradients: optimizer.zero_grad()
```

**Key difference from TensorFlow:**
- TensorFlow/Keras: High-level `fit()` handles everything
- PyTorch: Explicit control over each training step

In [ ]:
# Training hyperparameters
batch_size = 32
epochs = 10
learning_rate = 0.001
validation_split = 0.2

# Split training data into train and validation sets
num_train = int(len(x_train_spikes) * (1 - validation_split))
indices = torch.randperm(len(x_train_spikes))

train_indices = indices[:num_train]
val_indices = indices[num_train:]

x_train_split = x_train_spikes[train_indices]
y_train_split = y_train[train_indices]
x_val = x_train_spikes[val_indices]
y_val = y_train[val_indices]

# Create DataLoaders for batching
train_dataset = TensorDataset(x_train_split, y_train_split)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Initialize optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print("✓ Training setup complete!")
print(f"\nTraining samples: {len(x_train_split)}")
print(f"Validation samples: {len(x_val)}")
print(f"Batch size: {batch_size}")
print(f"Learning rate: {learning_rate}")
print(f"Epochs: {epochs}")

## Cell 10: Training Loop

**Translation Details:**
- `model.fit()` → Custom training loop
- Automatic progress bars → Manual progress tracking
- Built-in validation → Manual validation loop

**Significance:**
This is the core training loop that:
1. **Trains the model** using surrogate gradient descent
2. **Validates performance** on held-out data
3. **Tracks metrics** (loss and accuracy)

**What's happening during training:**
- **Forward pass:** Spikes propagate through network (LIF dynamics)
- **Loss computation:** Compare spike-based predictions to labels
- **Backward pass:** Gradients flow back using surrogate gradients!
- **Weight update:** Adjust synaptic weights to improve predictions

**The magic:** Even though forward pass uses discontinuous Heaviside function,
backward pass uses smooth surrogate gradient, enabling training!

In [ ]:
# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

print("Starting training...\n")
print("=" * 70)

# Training loop
for epoch in range(epochs):
    # ==================== Training Phase ====================
    model.train()  # Set model to training mode
    
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
        # Move data to device (GPU if available)
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        
        # Forward pass: Get spike predictions
        # x_batch: [batch, time_steps, input_dim]
        # outputs: [batch, time_steps, output_dim]
        outputs = model(x_batch)
        
        # Compute loss
        loss = spike_categorical_crossentropy(outputs, y_batch)
        
        # Backward pass: Compute gradients using surrogate gradient
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        # Track metrics
        train_loss += loss.item() * x_batch.size(0)
        
        # Compute accuracy
        spike_counts = torch.sum(outputs, dim=1)
        pred_class = torch.argmax(spike_counts, dim=-1)
        true_class = torch.argmax(y_batch, dim=-1)
        train_correct += (pred_class == true_class).sum().item()
        train_total += x_batch.size(0)
    
    # Calculate average training metrics
    train_loss = train_loss / train_total
    train_acc = 100.0 * train_correct / train_total
    
    # ==================== Validation Phase ====================
    model.eval()  # Set model to evaluation mode
    
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():  # Disable gradient computation for validation
        for x_batch, y_batch in val_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            
            # Forward pass
            outputs = model(x_batch)
            
            # Compute loss
            loss = spike_categorical_crossentropy(outputs, y_batch)
            val_loss += loss.item() * x_batch.size(0)
            
            # Compute accuracy
            spike_counts = torch.sum(outputs, dim=1)
            pred_class = torch.argmax(spike_counts, dim=-1)
            true_class = torch.argmax(y_batch, dim=-1)
            val_correct += (pred_class == true_class).sum().item()
            val_total += x_batch.size(0)
    
    # Calculate average validation metrics
    val_loss = val_loss / val_total
    val_acc = 100.0 * val_correct / val_total
    
    # Store history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Print epoch results
    print(f"Epoch {epoch+1}/{epochs}:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    print("-" * 70)

print("\n✓ Training complete!")
print(f"\nFinal validation accuracy: {history['val_acc'][-1]:.2f}%")

## Cell 11: Visualize Training History

**Significance:**
Plotting training curves helps us understand:
- **Convergence:** Is the model learning?
- **Overfitting:** Is validation performance degrading while training improves?
- **Stability:** Are the metrics smooth or noisy?

**What to look for:**
- Train loss should decrease over time
- Validation loss should track training loss (if not, overfitting)
- Accuracy should increase and plateau
- Gap between train/val suggests overfitting or underfitting

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot loss
ax1.plot(history['train_loss'], label='Training Loss', marker='o')
ax1.plot(history['val_loss'], label='Validation Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot accuracy
ax2.plot(history['train_acc'], label='Training Accuracy', marker='o')
ax2.plot(history['val_acc'], label='Validation Accuracy', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Training curves plotted!")

## Cell 12: Test Set Evaluation

**Significance:**
Final evaluation on held-out test set to assess generalization.
This tells us how well the SNN performs on completely unseen data.

In [ ]:
# Evaluate on test set
model.eval()

test_dataset = TensorDataset(x_test_spikes, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

test_loss = 0.0
test_correct = 0
test_total = 0

print("Evaluating on test set...\n")

with torch.no_grad():
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Forward pass
        outputs = model(x_batch)
        
        # Compute loss
        loss = spike_categorical_crossentropy(outputs, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        
        # Compute accuracy
        spike_counts = torch.sum(outputs, dim=1)
        pred_class = torch.argmax(spike_counts, dim=-1)
        true_class = torch.argmax(y_batch, dim=-1)
        test_correct += (pred_class == true_class).sum().item()
        test_total += x_batch.size(0)

test_loss = test_loss / test_total
test_acc = 100.0 * test_correct / test_total

print("=" * 50)
print("TEST SET RESULTS")
print("=" * 50)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.2f}%")
print("=" * 50)

## Summary: Key Translation Points

### 1. **Surrogate Gradient Function**
- **TensorFlow:** `@tf.custom_gradient` decorator
- **PyTorch:** `torch.autograd.Function` class with `forward()` and `backward()`

### 2. **Custom Layers**
- **TensorFlow:** Inherit from `keras.layers.Layer`, implement `build()` and `call()`
- **PyTorch:** Inherit from `nn.Module`, implement `__init__()` and `forward()`

### 3. **Dynamic Loops**
- **TensorFlow:** `tf.while_loop()` with `TensorArray`
- **PyTorch:** Regular Python `for` loops (dynamic graph!)

### 4. **Model Building**
- **TensorFlow:** Functional API with `keras.Input()` and `keras.Model()`
- **PyTorch:** Custom `nn.Module` with sequential layers

### 5. **Training**
- **TensorFlow:** High-level `model.compile()` and `model.fit()`
- **PyTorch:** Manual training loop with `optimizer.step()`

### Key Advantages of PyTorch Version:
1. **More explicit:** Easier to debug and understand what's happening
2. **More flexible:** Simpler to modify training loop or add custom behavior
3. **Dynamic graphs:** No need for `tf.TensorArray` or complex control flow
4. **Cleaner code:** Python loops instead of graph operations

### Preserved Functionality:
- ✓ Surrogate gradient training
- ✓ LIF neuron dynamics
- ✓ Rate-based spike encoding
- ✓ Spike-based classification
- ✓ All hyperparameters and architecture